In [1]:
import tkinter as tk
from tkinter import messagebox, Toplevel, Label
from tkinter import ttk
import cv2
import mysql.connector
import os
from PIL import Image, ImageTk
import numpy as np
import glob

In [3]:
# Create the main window
window = tk.Tk()
window.title("Facial Recognition System")
window.geometry("800x600")
window.configure(bg='lightgrey')

# Create a style
style = ttk.Style()
style.configure("TLabel", font=("Arial", 14), background='lightgrey')
style.configure("TButton", font=("Arial", 14), padding=10)
style.configure("TEntry", font=("Arial", 14))

# Create frames for better organization
input_frame = ttk.Frame(window, padding="10 10 10 10")
input_frame.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S))

button_frame = ttk.Frame(window, padding="10 10 10 10")
button_frame.grid(column=0, row=1, sticky=(tk.W, tk.E, tk.N, tk.S))

# Input fields
ttk.Label(input_frame, text="Name").grid(column=0, row=0, sticky=tk.W, padx=5, pady=5)
t1 = ttk.Entry(input_frame, width=50)
t1.grid(column=1, row=0, padx=5, pady=5)

ttk.Label(input_frame, text="Age").grid(column=0, row=1, sticky=tk.W, padx=5, pady=5)
t2 = ttk.Entry(input_frame, width=50)
t2.grid(column=1, row=1, padx=5, pady=5)

ttk.Label(input_frame, text="Address").grid(column=0, row=2, sticky=tk.W, padx=5, pady=5)
t3 = ttk.Entry(input_frame, width=50)
t3.grid(column=1, row=2, padx=5, pady=5)

# Function to train the classifier
def train_classifier():
    data_dir = "C:/Users/HP/Desktop/Apps/Facial-Recog-Software/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')  # Convert image to grayscale
        imageNp = np.array(img, 'uint8')        
        
        # Extract ID from filename
        id = int(os.path.split(image)[1].split('_')[1].split('.')[0])

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids, dtype=np.int32)
    
    # Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result', 'Training Dataset completed!!!')

# Buttons
ttk.Button(button_frame, text="Train", command=train_classifier).grid(column=0, row=0, padx=5, pady=5)

def detect_face():
    def draw_boundray(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
        
        coords = []
        
        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)
            id, pred = clf.predict(gray_image[y:y + h, x:x + w])
            confidence = int(100 * (1 - pred / 300))
            
            mydb = mysql.connector.connect(
                host="localhost",
                user="root",
                password="",
                database="Authorixed_user"
            )
            mycursor = mydb.cursor()
            mycursor.execute("SELECT name FROM my_table WHERE id=%s", (id,))
            s = mycursor.fetchone()
            s = '' + ''.join(s) if s else "UNKNOWN"
            
            if confidence > 80:
                cv2.putText(img, s, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                # Log attendance
                mycursor.execute("INSERT INTO attendance (user_id, name) VALUES (%s, %s)", (id, s))
                mydb.commit()
            else:
                cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
                # Save unknown face
                unknown_dir = "unknown"
                if not os.path.exists(unknown_dir):
                    os.makedirs(unknown_dir)
                face = img[y:y + h, x:x + w]
                face = cv2.resize(face, (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = os.path.join(unknown_dir, f"unknown_{x}_{y}.jpg")
                cv2.imwrite(file_name_path, face)
                
            coords = [x, y, w, h]
        return coords

    def recognize(img, clf, faceCascade):
        coords = draw_boundray(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
        return img 

    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf, faceCascade)
        cv2.imshow("Face Detection", img)
        
        if cv2.waitKey(1) == 13:
            break
        
    video_capture.release()
    cv2.destroyAllWindows()

# Automatically start face detection
detect_face()

ttk.Button(button_frame, text="Detect Face", command=detect_face).grid(column=1, row=0, padx=5, pady=5)

def generate_dataset():
    if t1.get() == "" or t2.get() == "" or t3.get() == "":
        messagebox.showinfo('Result', 'Please provide complete details of the user')
    else:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="Authorixed_user"
        )
        mycursor = mydb.cursor()
        mycursor.execute("SELECT * from my_table")
        myresult = mycursor.fetchall()
        id = 1
        for x in myresult:
            id += 1
        sql = "INSERT INTO my_table(id, Name, Age, Address) VALUES (%s, %s, %s, %s)"
        val = (id, t1.get(), t2.get(), t3.get())
        mycursor.execute(sql, val)
        mydb.commit()
        
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)

            if len(faces) == 0:
                return None
            for (x, y, w, h) in faces:
                cropped_face = img[y:y + h, x:x + w]
            return cropped_face

        cap = cv2.VideoCapture(0)
        directory = "data"
        if not os.path.exists(directory):  # Check if directory exists
            os.makedirs(directory)  # Create directory if it doesn't exist

        # Find the highest user ID in the filenames of the existing images
        files = glob.glob(f"{directory}/user_*.jpg")
        if files:
            user_id = max(int(file.split('_')[1].split('.')[0]) for file in files)
        else:
            user_id = 0

        user_id += 1  # Increment the user ID for the new images

        images_generated = 0  # New variable to count the number of images generated

        while True:
            ret, frame = cap.read()
            if face_cropped(frame) is not None:
                images_generated += 1  # Increment the count each time an image is generated
                face = cv2.resize(face_cropped(frame), (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = os.path.join(directory, f"user_{user_id}_{images_generated}.jpg")
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(images_generated), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

                cv2.imshow("Cropped face", face)
                if cv2.waitKey(1) == 13 or images_generated == 50:  # Check if 50 images have been generated
                    break
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result', 'Generating Dataset completed!!!')

ttk.Button(button_frame, text="Generate Data", command=generate_dataset).grid(column=2, row=0, padx=5, pady=5)

def view_attendance():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="Authorixed_user"
    )
    mycursor = mydb.cursor()
    mycursor.execute("SELECT * FROM attendance")
    records = mycursor.fetchall()
    
    attendance_window = tk.Toplevel(window)
    attendance_window.title("Attendance Records")
    
    text_area = tk.Text(attendance_window, wrap='word', font=("Arial", 12))
    text_area.pack(expand=True, fill='both')
    
    for record in records:
        text_area.insert(tk.END, f"ID: {record[0]}, Name:{record[1]}, Timestamp: {record[2]}\n")
        
ttk.Button(button_frame, text="View Attendance", command=view_attendance).grid(column=3, row=0, padx=5, pady=5)

# Function to show person image

def update_name(person_id, entry, label):
    new_name = entry.get()
    if new_name:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="Authorixed_user"
        )
        mycursor = mydb.cursor()
        sql = "UPDATE my_table SET name = %s WHERE id = %s"
        val = (new_name, person_id)
        mycursor.execute(sql, val)
        mydb.commit()
        label.config(text=new_name)
        messagebox.showinfo('Result', 'Name updated successfully')

def show_person_image():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="Authorixed_user"
    )
    mycursor = mydb.cursor()
    mycursor.execute("SELECT id, name FROM my_table")
    records = mycursor.fetchall()
    
    if not records:
        messagebox.showinfo('Result', 'No records found')
        return
    
    person_window = Toplevel(window)
    person_window.title("Person Image")
    
    for record in records:
        person_id, person_name = record
        image_path = f"data/user_{person_id}_1.jpg"  # Assuming the first image is used
        
        if os.path.exists(image_path):
            img = Image.open(image_path)
            img = img.resize((200, 200), Image.LANCZOS)
            img = ImageTk.PhotoImage(img)
            
            panel = Label(person_window, image=img)
            panel.image = img  # Keep a reference to avoid garbage collection
            panel.pack()
            
            name_label = Label(person_window, text=person_name if person_name else "Unknown", font=("Arial", 14))
            name_label.pack()
        else:
            # Display a placeholder for unknown images
            placeholder_img = Image.new('RGB', (200, 200), color = (73, 109, 137))
            img = ImageTk.PhotoImage(placeholder_img)
            
            panel = Label(person_window, image=img)
            panel.image = img  # Keep a reference to avoid garbage collection
            panel.pack()
            
            name_label = Label(person_window, text="Unknown", font=("Arial", 14))
            name_label.pack()

# Assuming `window` and `button_frame` are already defined
ttk.Button(button_frame, text="Show Person Image", command=show_person_image).grid(column=4, row=0, padx=5, pady=5)

# Start the main loop
window.mainloop()